<a href="https://colab.research.google.com/github/LoredoJ08/mcd_dm_jlg/blob/main/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tarea 2 (10 puntos). Operaciones con RDDs

- Obtener información de algún origen de datos (propio o de API)
- Convertir el origen de datos a RDD con PySpark
- Realizar alguna operación en el RDD, como estadísticas descriptivas
  básicas
- Subir la práctica a un repositorio público y etiquetarla claramente


In [1]:
#- Instalar en un entorno local o ejecutar Spark en algún servidor en línea (como Google Colab)
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F


spark= SparkSession.builder.appName("Intento 1: Spark").getOrCreate()

#https://github.com/LoredoJ08/mcd_dm_jlg.git

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,590 kB]
Get:14 http://security.ubuntu.com

In [2]:
import os
import urllib.request


In [3]:
url = 'https://raw.githubusercontent.com/LoredoJ08/mcd_dm_jlg/main/Tarea%201/diabetes_dataset00.csv'
file_name = 'diabetes_dataset00.csv'

# CSV
urllib.request.urlretrieve(url, file_name)
df_spark = spark.read.csv(file_name, header=True, inferSchema=True)
df_spark.show()


+--------------------+---------------+--------------+--------------+---------------------+--------------+---+---+-----------------+--------------+--------------+------------------+-------------------+--------------------+---------+---------------------+--------------+-------------------+----------------------+---------------+-----------------------------+-----------------+----------------------------+-----------------+------------------+-------------------------+-------------------+---------------+------------------------+--------------------+-----------------------+---------------+------------+--------------------+
|              Target|Genetic Markers|Autoantibodies|Family History|Environmental Factors|Insulin Levels|Age|BMI|Physical Activity|Dietary Habits|Blood Pressure|Cholesterol Levels|Waist Circumference|Blood Glucose Levels|Ethnicity|Socioeconomic Factors|Smoking Status|Alcohol Consumption|Glucose Tolerance Test|History of PCOS|Previous Gestational Diabetes|Pregnancy History|W

In [4]:
# Mostrar el esquema de los datos
df_spark.printSchema()


root
 |-- Target: string (nullable = true)
 |-- Genetic Markers: string (nullable = true)
 |-- Autoantibodies: string (nullable = true)
 |-- Family History: string (nullable = true)
 |-- Environmental Factors: string (nullable = true)
 |-- Insulin Levels: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- BMI: integer (nullable = true)
 |-- Physical Activity: string (nullable = true)
 |-- Dietary Habits: string (nullable = true)
 |-- Blood Pressure: integer (nullable = true)
 |-- Cholesterol Levels: integer (nullable = true)
 |-- Waist Circumference: integer (nullable = true)
 |-- Blood Glucose Levels: integer (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- Socioeconomic Factors: string (nullable = true)
 |-- Smoking Status: string (nullable = true)
 |-- Alcohol Consumption: string (nullable = true)
 |-- Glucose Tolerance Test: string (nullable = true)
 |-- History of PCOS: string (nullable = true)
 |-- Previous Gestational Diabetes: string (nullable = t

In [5]:
# Convertir DataFrame a RDD
rdd = df_spark.rdd

# Seleccionar columnas 'Blood Glucose Levels' y 'BMI'
rdd_glucose_bmi = df_spark.select('Blood Glucose Levels', 'BMI').rdd



In [9]:
# Convertir RDD de columnas a valores
rdd_glucose_bmi_clean = rdd_glucose_bmi.map(lambda row: (float(row['Blood Glucose Levels']), float(row['BMI']))) \
                                       .filter(lambda x: x[0] is not None and x[1] is not None)

# 1.-transformamos los datos
rdd_glucose = rdd_glucose_bmi_clean.map(lambda x: x[0])
rdd_bmi = rdd_glucose_bmi_clean.map(lambda x: x[1])

#datos en lista
glucose_data = rdd_glucose.collect()
bmi_data = rdd_bmi.collect()

#calcular las estadísticas
def calculate_manual_stats(data):
    count = len(data)
    total = sum(data)
    mean = total / count

    sum_squared_diff = sum((x - mean) ** 2 for x in data)
    variance = sum_squared_diff / count

    return count, mean, variance

#estadísticas para los niveles de glucosa y BMI (indice de masa corporal)
glucose_stats = calculate_manual_stats(glucose_data)
bmi_stats = calculate_manual_stats(bmi_data)


print(f"Niveles de glucosa - Count: {glucose_stats[0]}, Mean: {glucose_stats[1]}, Variance: {glucose_stats[2]}")
print(f"BMI - Count: {bmi_stats[0]}, Mean: {bmi_stats[1]}, Variance: {bmi_stats[2]}")


Niveles de glucosa - Count: 70000, Mean: 160.70165714285713, Variance: 2319.886820111044
BMI - Count: 70000, Mean: 24.782942857142856, Variance: 36.170514768162185
